In [42]:
# json 파일 경로 지정
split_file = '/data/ephemeral/home/Jungyeon/level2-cv-semanticsegmentation-cv-02-lv3/Data/train_valid_split/splits/fold_0.json'
image_root = "/data/ephemeral/home/data/train/DCM"
label_root = "/data/ephemeral/home/data/train/outputs_json"

with open(split_file, 'r') as f:
    split_data = json.load(f)
    train_dataset = split_data['train_filenames']
    train_labelnames = split_data['train_labelnames']
    valid_dataset = split_data['valid_filenames']
    valid_labelnames = split_data['valid_labelnames']


In [138]:
import os
import shutil

def move_images(destination_path, dataset, image_root):
    
    # 경로 생성
    os.makedirs(destination_path, exist_ok=True)

    # 파일 이동
    for file_path in dataset:
        new_file_path = file_path[6:]
        source_file = os.path.join(image_root, file_path)
        destination_file = os.path.join(destination_path, new_file_path)
        shutil.copy(source_file, destination_file)  # 파일 복사
            
    print("파일 이동 완료!")

# move_images("/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/images/train/", train_dataset, image_root)
# move_images("/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/images/valid/", valid_dataset, image_root)
move_images("/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/labels_base/train/", train_labelnames, label_root)
move_images("/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/labels_base/valid/", valid_labelnames, label_root)


파일 이동 완료!
파일 이동 완료!


In [150]:
import os

# 경로 설정
train_folder = '/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/labels/valid'  # train 폴더의 절대경로 또는 상대경로

# 파일 개수 세기
file_count = 0
for root, dirs, files in os.walk(train_folder):
    file_count += len(files)  # 현재 디렉토리 안의 파일 개수 추가

print(f"Train 폴더 안의 파일 개수: {file_count}")

Train 폴더 안의 파일 개수: 160


In [120]:
# class-index to number index
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]

CLASS2IND = {v: i for i, v in enumerate(CLASSES)}

IND2CLASS = {v: k for k, v in CLASS2IND.items()}
print(IND2CLASS)

{0: 'finger-1', 1: 'finger-2', 2: 'finger-3', 3: 'finger-4', 4: 'finger-5', 5: 'finger-6', 6: 'finger-7', 7: 'finger-8', 8: 'finger-9', 9: 'finger-10', 10: 'finger-11', 11: 'finger-12', 12: 'finger-13', 13: 'finger-14', 14: 'finger-15', 15: 'finger-16', 16: 'finger-17', 17: 'finger-18', 18: 'finger-19', 19: 'Trapezium', 20: 'Trapezoid', 21: 'Capitate', 22: 'Hamate', 23: 'Scaphoid', 24: 'Lunate', 25: 'Triquetrum', 26: 'Pisiform', 27: 'Radius', 28: 'Ulna'}


In [131]:
def normalize_coordinates(data, image_width, image_height):
    """Normalize coordinates and ensure they are within bounds."""
    normalized_lines = []
    for line in data:
        normalized_numbers = []
        for i, num in enumerate(line):
            if i % 2 == 0:  # x coordinate
                normalized_num = num / image_width
            else:  # y coordinate
                normalized_num = num / image_height

            # Ensure coordinates are within [0, 1]
            normalized_num = max(0.0, min(1.0, normalized_num))
            normalized_numbers.append(normalized_num)

        normalized_lines.append(" ".join(map(str, normalized_numbers)))
    return normalized_lines

In [ ]:
# YOLO : <class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>

import json

json_dir = "/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/labels_base/train"
yolo_dir = '/data/ephemeral/home/Jungyeon/Yolo/datasets/base-seg/labels/train'

for json_path in os.listdir(json_dir):
    print(json_path)

    annotation_path = os.path.join(json_dir, json_path)
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)

    for annotation in annotations['annotations']:
        yolo_ann = ""
        
        class_label = annotation['label']
        class_idx = CLASS2IND[class_label]
        
        yolo_ann += str(class_idx) + " "
        
        points = annotation['points']
        
        norm_points = normalize_coordinates(points, 2048, 2048)

        for point in norm_points:
            yolo_ann += " ".join(str(p) for p in point.split()) + " "
     
        # 파일 경로 설정
        file_path = f'{yolo_dir}/{json_path[:-4]}txt'
        
        # 파일 경로의 디렉토리가 없다면 디렉토리 생성
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            os.makedirs(directory)

        # 파일에 데이터 추가
        with open(file_path, "a") as file:  # "a" 모드는 append
            line = yolo_ann + "\n"
            file.write(line)

        print(f"Data successfully added to {file_path}")